# Exercise: One-hot vector

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is useful in machine learning, as we'll see later.

It simply means that among a group of bits (all either 0 or 1) only one is hot (1) while all others are cold (0),

`v = [0, 0, 0, 0, 0, 1, 0, 0, 0]`

### Task

1. Think about what information an implementation of a one-hot vector actually has to store.
2. Define a `OneHot` type which represents a vector with only a single hot (i.e. `== 1`) bit.
3. Extend all the necessary `Base` functions such that the following computation works for a matrix `A` and a vector of `OneHot` vectors `vs` (i.e. `vs isa Vector{OneHot}`).

    ```julia
    function innersum(A, vs)
        t = zero(eltype(A)) # generic!
        for v in vs
            y = A*v
            for i in 1:length(vs[1])
                t += v[i] * y[i]
            end
        end
        return t
    end

    A = rand(3,3)
    vs = [rand(3) for _ = 1:10] # This should be replaced by a `Vector{OneHot}`

    innersum(A, vs)

    ```

4. Benchmark the speed of `innersum` when called with a `OneHot` vector or with a `Vector{Float64}`, respectively.
 * Do you observe a speed up?


5. Now, define a `OneHotVector` type which is identical to `OneHot` but is declared to be a subtype of `AbstractVector{Bool}` and extend only the functions `Base.getindex(v::OneHotVector, i::Int)` and `Base.size(v::OneHotVector)`.
 * Here, the function `size` should return a `Tuple{Int64}` indicating the length of the vector, i.e. `(3,)` for a one-hot vector of length 3 (see the [AbstractArray interface](https://docs.julialang.org/en/latest/manual/interfaces/#man-interface-array-1) for more information)
 

6. Try to create a single `OneHotVector` and try to run the `innersum` function using the new `OneHotVector` type.
 * What changes do you observe?
 * Do you have to implement any further methods?

In [1]:
function innersum(A, vs)
     t = zero(eltype(A)) # generic!
     for v in vs
         y = A*v
         for i in 1:length(vs[1])
             t += v[i] * y[i]
         end
     end
     return t
 end

innersum (generic function with 1 method)

### Possible solution

1. It must store it's length and the position of the hot bit.

In [2]:
# 2.
# One-hot vector
# v = (0, ..., 0, 1, 0, .., 0)
struct OneHot
    len::Int
    ind::Int
end

In [3]:
# 3.
import Base: *, getindex, length

length(v::OneHot) = v.len
getindex(v::OneHot, i::Int) = i == v.ind
*(A::AbstractMatrix, v::OneHot) = A[:, v.ind]

* (generic function with 347 methods)

In [4]:
# 4.
using BenchmarkTools
A = rand(3,3)
vs = [OneHot(3, rand(1:3)) for _ in 1:10]
vs_float64 = [rand(3) for _ in 1:10]

@btime innersum($A, $vs);
@btime innersum($A, $vs_float64);

  318.459 ns (10 allocations: 1.09 KiB)
  720.596 ns (10 allocations: 1.09 KiB)


 I see about an 0.5x speed up!

In [5]:
# 5.
struct OneHotVector <: AbstractVector{Bool}
    len::Int
    ind::Int
end

Base.size(v::OneHotVector) = (v.len,)
Base.getindex(v::OneHotVector, i::Integer) = i == v.ind

In [6]:
# 6.

# Automatically, we get a nice visualization
OneHotVector(4, 2)

4-element OneHotVector:
 false
  true
 false
 false

In [7]:
# Also, innersum just works.
A = rand(3,3)
vs = [OneHotVector(3, rand(1:3)) for _ in 1:10]

innersum(A, vs)

7.36674772788775